# Exporting ATT&CK Group Navigator Layers

# Get Relationship STIX Objects - (Manual)
-----------------------
I believe it is important to understand the code behind the main functions available in the Python library [attackcti](https://attackcti.readthedocs.io/en/latest/index.html). I highly recommend to first read the docs I put together about [cti-taxii-client](https://attackcti.readthedocs.io/en/latest/taxii_client.html) and [cti-python-stix2](https://attackcti.readthedocs.io/en/latest/stix.html) libraries.
Those two summarize several of the concepts that I had to read to understand how to perform a simple query against ATT&CK's TAXII server

## Import STIX and TAXII Libraries

In [1]:
from stix2 import TAXIICollectionSource, Filter, CompositeDataSource
from taxii2client.v20 import Collection

## Set ATT&CK TAXII Collection ID Variables
The public ATT&CK TAXII instance has three main collections (Enterprise, Pre and Mobile). Every collection has an ID which attackcti uses to retrieve ATT&CK STIX objects from all those matrices.

In [2]:
ATTACK_STIX_COLLECTIONS = "https://cti-taxii.mitre.org/stix/collections/"
ENTERPRISE_ATTACK = "95ecc380-afe9-11e4-9b6c-751b66dd541e"
PRE_ATTACK = "062767bd-02d2-4b72-84ba-56caef0f8658"
MOBILE_ATTACK = "2f669986-b40b-4423-b720-4396ca6a462b"
ICS_ATTACK = "02c3ef24-9cd4-48f3-a99f-b74ce24f1d34"

## Initialize TAXII Collection Sources
According to [STIX2 docs](https://stix2.readthedocs.io/en/latest/index.html), the [TAXIICollectionSource API](https://stix2.readthedocs.io/en/latest/api/datastore/stix2.datastore.taxii.html#stix2.datastore.taxii.TAXIICollectionSource) provides an interface for searching/retrieving STIX objects from a local/remote TAXII Collection endpoint. In our case, we are pointing to our ATT&CK TAXII Collection instances (https://cti-taxii.mitre.org/stix/collections/<Collection ID>)

In [3]:
ENTERPRISE_COLLECTION = Collection(ATTACK_STIX_COLLECTIONS + ENTERPRISE_ATTACK + "/")
TC_ENTERPRISE_SOURCE = TAXIICollectionSource(ENTERPRISE_COLLECTION)
PRE_COLLECTION = Collection(ATTACK_STIX_COLLECTIONS + PRE_ATTACK + "/")
TC_PRE_SOURCE = TAXIICollectionSource(PRE_COLLECTION)
MOBILE_COLLECTION = Collection(ATTACK_STIX_COLLECTIONS + MOBILE_ATTACK + "/")
TC_MOBILE_SOURCE = TAXIICollectionSource(MOBILE_COLLECTION)
ICS_COLLECTION = Collection(ATTACK_STIX_COLLECTIONS + ICS_ATTACK + "/")
TC_ICS_SOURCE = TAXIICollectionSource(ICS_COLLECTION)

## Initialize a Composite Data Source
According to [STIX2 docs](https://stix2.readthedocs.io/en/latest/index.html), a user can have a single [CompositeDataSource](https://stix2.readthedocs.io/en/latest/api/stix2.datastore.html#stix2.datastore.CompositeDataSource) as an interface to a set of DataSources. When an API call is made to the CompositeDataSource, it is delegated to each of the (real) DataSources that are attached to it. In our case, we have three TAXIICollection sources (Enterprise, PRE and Mobile) as defined in our previous step. Therefore, we can use the CompositeDataSource class and the add_data_sources method to attach every ATT&CK TAXIICollection source and be able to query all of them at the same time.

In [4]:
COMPOSITE_DS = CompositeDataSource()
COMPOSITE_DS.add_data_sources([TC_ENTERPRISE_SOURCE, TC_PRE_SOURCE, TC_MOBILE_SOURCE, TC_ICS_SOURCE])

## Retrieve all relationships
Now that we can query all the ATT&CK TAXIICollection sources at once, we can use the query method and a set of filters to retrieve STIX objects of type relationship

In [5]:
rels = COMPOSITE_DS.query(Filter("type", "=", "relationship"))
rels[0]

Relationship(type='relationship', id='relationship--fbe555c3-5c7b-44e7-a48f-293bdae9de0c', created_by_ref='identity--c78cb6e5-0c4b-4611-8297-d1b8b55e40b5', created='2021-04-27T15:46:45.720Z', modified='2021-04-27T15:46:45.720Z', relationship_type='uses', description="[AppleJeus](https://attack.mitre.org/software/S0584)'s spearphishing links required user interaction to navigate to the malicious website.(Citation: CISA AppleJeus Feb 2021)", source_ref='malware--e2d34c63-6f5a-41f5-86a2-e2380f27f858', target_ref='attack-pattern--ef67e13e-5598-4adc-bdb2-998225874fa9', external_references=[ExternalReference(source_name='CISA AppleJeus Feb 2021', description='Cybersecurity and Infrastructure Security Agency. (2021, February 21). AppleJeus: Analysis of North Korea’s Cryptocurrency Malware. Retrieved March 1, 2021.', url='https://us-cert.cisa.gov/ncas/alerts/aa21-048a')], object_marking_refs=['marking-definition--fa42a846-8d90-4e51-bc29-71d5b4802168'])

## Retrieve all relationships from an specific STIX object
What if you want to be very specific and get relationships from a specific STIX objects? You can use the [relationships](https://stix2.readthedocs.io/en/latest/api/stix2.datastore.html#stix2.datastore.CompositeDataSource.relationships) method from the [CompositeDataSource](https://stix2.readthedocs.io/en/latest/api/stix2.datastore.html#stix2.datastore.CompositeDataSource) class to retrieve relationships involving a given STIX object.

In [6]:
from attackcti import attack_client
lift = attack_client()

groups = lift.get_groups()
groups = lift.remove_revoked(groups)

rels = COMPOSITE_DS.relationships(groups[0], 'uses', source_only=True)
rels[0]

Relationship(type='relationship', id='relationship--12e483aa-14a0-41ea-b6fd-7ced3590472b', created_by_ref='identity--c78cb6e5-0c4b-4611-8297-d1b8b55e40b5', created='2021-04-14T14:05:51.798Z', modified='2021-04-14T14:05:51.798Z', relationship_type='uses', description='(Citation: Check Point Rocket Kitten)', source_ref='intrusion-set--fa19de15-6169-428d-9cd6-3ca3d56075b7', target_ref='tool--fbd727ea-c0dc-42a9-8448-9e12962d1ab5', external_references=[ExternalReference(source_name='Check Point Rocket Kitten', description='Check Point Software Technologies. (2015). ROCKET KITTEN: A CAMPAIGN WITH 9 LIVES. Retrieved March 16, 2018.', url='https://blog.checkpoint.com/wp-content/uploads/2015/11/rocket-kitten-report.pdf')], object_marking_refs=['marking-definition--fa42a846-8d90-4e51-bc29-71d5b4802168'])

# Get Relationship STIX Objects - (Automatic)
-----------------------

## Retrieve all relationships

In [7]:
from attackcti import attack_client
lift = attack_client()

In [8]:
%time all_relationships = lift.get_relationships()

CPU times: user 3.04 s, sys: 74.9 ms, total: 3.11 s
Wall time: 5.49 s


In [9]:
all_relationships[0]

Relationship(type='relationship', id='relationship--fbe555c3-5c7b-44e7-a48f-293bdae9de0c', created_by_ref='identity--c78cb6e5-0c4b-4611-8297-d1b8b55e40b5', created='2021-04-27T15:46:45.720Z', modified='2021-04-27T15:46:45.720Z', relationship_type='uses', description="[AppleJeus](https://attack.mitre.org/software/S0584)'s spearphishing links required user interaction to navigate to the malicious website.(Citation: CISA AppleJeus Feb 2021)", source_ref='malware--e2d34c63-6f5a-41f5-86a2-e2380f27f858', target_ref='attack-pattern--ef67e13e-5598-4adc-bdb2-998225874fa9', external_references=[ExternalReference(source_name='CISA AppleJeus Feb 2021', description='Cybersecurity and Infrastructure Security Agency. (2021, February 21). AppleJeus: Analysis of North Korea’s Cryptocurrency Malware. Retrieved March 1, 2021.', url='https://us-cert.cisa.gov/ncas/alerts/aa21-048a')], object_marking_refs=['marking-definition--fa42a846-8d90-4e51-bc29-71d5b4802168'])

## Retrieve all relationships from an specific STIX object

In [10]:
groups = lift.get_groups()
groups = lift.remove_revoked(groups)

In [11]:
%time group_relationships = lift.get_relationships_by_object(groups[0])

CPU times: user 357 ms, sys: 53.8 ms, total: 411 ms
Wall time: 3.91 s


In [12]:
group_relationships[0]

Relationship(type='relationship', id='relationship--12e483aa-14a0-41ea-b6fd-7ced3590472b', created_by_ref='identity--c78cb6e5-0c4b-4611-8297-d1b8b55e40b5', created='2021-04-14T14:05:51.798Z', modified='2021-04-14T14:05:51.798Z', relationship_type='uses', description='(Citation: Check Point Rocket Kitten)', source_ref='intrusion-set--fa19de15-6169-428d-9cd6-3ca3d56075b7', target_ref='tool--fbd727ea-c0dc-42a9-8448-9e12962d1ab5', external_references=[ExternalReference(source_name='Check Point Rocket Kitten', description='Check Point Software Technologies. (2015). ROCKET KITTEN: A CAMPAIGN WITH 9 LIVES. Retrieved March 16, 2018.', url='https://blog.checkpoint.com/wp-content/uploads/2015/11/rocket-kitten-report.pdf')], object_marking_refs=['marking-definition--fa42a846-8d90-4e51-bc29-71d5b4802168'])

# Retrive Techniques used by one Group - (Manual)
-----------------------

In this case we want relationship objects that have target_ref values of type attack-pattern. Following the manual code I shared above, and the results from the `get_relationships_by_object()` function, you can simply query the ATT&CK Enterprise TAXIICollection source with the filter below

In [13]:
filter_objects = [
  Filter('type', '=', 'attack-pattern'),
  Filter('id', '=', [r.target_ref for r in group_relationships])
]
techniques_used = TC_ENTERPRISE_SOURCE.query(filter_objects)
techniques_used[0]

AttackPattern(type='attack-pattern', id='attack-pattern--232b7f21-adf9-4b42-b936-b9d6f7df856e', created_by_ref='identity--c78cb6e5-0c4b-4611-8297-d1b8b55e40b5', created='2020-03-11T14:49:36.954Z', modified='2020-03-11T14:55:56.177Z', name='Malicious File', description="An adversary may rely upon a user opening a malicious file in order to gain execution. Users may be subjected to social engineering to get them to open a file that will lead to code execution. This user action will typically be observed as follow-on behavior from [Spearphishing Attachment](https://attack.mitre.org/techniques/T1566/001). Adversaries may use several types of files that require a user to execute them, including .doc, .pdf, .xls, .rtf, .scr, .exe, .lnk, .pif, and .cpl.\n\nAdversaries may employ various forms of [Masquerading](https://attack.mitre.org/techniques/T1036) on the file to increase the likelihood that a user will open it.\n\nWhile [Malicious File](https://attack.mitre.org/techniques/T1204/002) freq

# Retrive Techniques used by one Group - (Automatic)
-----------------------

In [14]:
from attackcti import attack_client
lift = attack_client()
groups = lift.get_groups()
groups = lift.remove_revoked(groups)
group_techniques = lift.get_techniques_used_by_group(groups[0])
group_techniques[0]

AttackPattern(type='attack-pattern', id='attack-pattern--232b7f21-adf9-4b42-b936-b9d6f7df856e', created_by_ref='identity--c78cb6e5-0c4b-4611-8297-d1b8b55e40b5', created='2020-03-11T14:49:36.954Z', modified='2020-03-11T14:55:56.177Z', name='Malicious File', description="An adversary may rely upon a user opening a malicious file in order to gain execution. Users may be subjected to social engineering to get them to open a file that will lead to code execution. This user action will typically be observed as follow-on behavior from [Spearphishing Attachment](https://attack.mitre.org/techniques/T1566/001). Adversaries may use several types of files that require a user to execute them, including .doc, .pdf, .xls, .rtf, .scr, .exe, .lnk, .pif, and .cpl.\n\nAdversaries may employ various forms of [Masquerading](https://attack.mitre.org/techniques/T1036) on the file to increase the likelihood that a user will open it.\n\nWhile [Malicious File](https://attack.mitre.org/techniques/T1204/002) freq

# Retrive all Techniques used by all Groups - (Manual)
-----------------------
You can apply the same get_techniques_used_by_group() function, but against all the groups STIX objects that the get_groups() function retrieves. You can do a simple for loop over more than 90 groups. However, it takes longer than what I would like it to take. Therefore, I decided to go a different route and started testing some code.

## Get all groups and techniques

In [15]:
from attackcti import attack_client
lift = attack_client()
groups = lift.get_groups()
techniques = lift.get_techniques()
techniques = lift.remove_revoked(techniques)

## Filter Group objects using techniques

In [16]:
from stix2.utils import get_type_from_id
group_relationships = []
for rel in all_relationships:
    if get_type_from_id(rel.source_ref) == 'intrusion-set'\
    and get_type_from_id(rel.target_ref) == 'attack-pattern':
        group_relationships.append(rel)
len(group_relationships)
print(group_relationships[0])

{
    "type": "relationship",
    "id": "relationship--435c288d-1e10-4610-bf41-531390e5a650",
    "created_by_ref": "identity--c78cb6e5-0c4b-4611-8297-d1b8b55e40b5",
    "created": "2021-04-25T23:26:10.590Z",
    "modified": "2021-04-25T23:26:10.590Z",
    "relationship_type": "uses",
    "description": "[MuddyWater](https://attack.mitre.org/groups/G0069) has used JavaScript files to execute its [POWERSTATS](https://attack.mitre.org/software/S0223) payload.(Citation: ClearSky MuddyWater Nov 2018)(Citation: FireEye MuddyWater Mar 2018)",
    "source_ref": "intrusion-set--269e8108-68c6-4f99-b911-14b2e765dec2",
    "target_ref": "attack-pattern--0f4a0c76-ab2d-4cb0-85d3-3f0efb8cba0d",
    "external_references": [
        {
            "source_name": "ClearSky MuddyWater Nov 2018",
            "description": "ClearSky Cyber Security. (2018, November). MuddyWater Operations in Lebanon and Oman: Using an Israeli compromised domain for a two-stage campaign. Retrieved November 29, 2018.",
     

## Match Group -> Relationships Intrusion-set ID
Then, I just take all the group_relationships I got, and look for the specific `intrusion-set (Group)` id in the groups STIX objects. Once there is a match, I create new fields on the `intrusion-set (Group)` STIX object to add additional information about the `attack-pattern (Technique)` in the relationship object. The most important additional metadata is the target_ref field which points to the specific `attack-pattern (Technique)` id involving the `group`. The results are then added to a new list named `group_techniques_ref` .

In [17]:
import json
group_techniques_ref = []
for g in groups:
    for rel in group_relationships:
        if g['id'] == rel['source_ref']:
            gs = json.loads(g.serialize())
            gs
            gs['technique_ref'] = rel['target_ref']
            gs['relationship_description'] = rel['description']
            gs['relationship_id'] = rel['id']
            group_techniques_ref.append(gs)

## Match Attack-patterns -> Intrusion-set object ID
I apply the same concept as before, and just loop through all the attack-pattern objects and look for the specific attack-pattern id in the initial relationships STIX objects. Once there is a match, I add additional information from the attack-pattern itself to the python dictionaries in the `group_techniques_ref` list. The results then get added to a new list named `groups_use_techniques`.

In [18]:
groups_use_techniques = []
for gt in group_techniques_ref:
    for t in techniques:
        if gt['technique_ref'] == t['id']:
            tactic_list = list()
            for phase in t['kill_chain_phases']:
                tactic_list.append(phase['phase_name'])
                gt['technique'] = t['name']
                gt['technique_description'] = t['description']
                gt['tactic'] = tactic_list
                gt['technique_id'] = t['external_references'][0]['external_id']
                gt['matrix'] =  t['external_references'][0]['source_name']
                if 'x_mitre_platforms' in t.keys():
                    gt['platform'] = t['x_mitre_platforms']
                if 'x_mitre_data_sources' in t.keys():
                    gt['data_sources'] = t['x_mitre_data_sources']
                if 'x_mitre_permissions_required' in t.keys():
                    gt['permissions_required'] = t['x_mitre_permissions_required']
                if 'x_mitre_effective_permissions' in t.keys():
                    gt['effective_permissions'] = t['x_mitre_effective_permissions']
                groups_use_techniques.append(gt)
groups_use_techniques[0]

{'created_by_ref': 'identity--c78cb6e5-0c4b-4611-8297-d1b8b55e40b5',
 'object_marking_refs': ['marking-definition--fa42a846-8d90-4e51-bc29-71d5b4802168'],
 'external_references': [{'external_id': 'G0130',
   'source_name': 'mitre-attack',
   'url': 'https://attack.mitre.org/groups/G0130'},
  {'source_name': 'Operation Woolen-Goldfish',
   'description': 'Analysis of infrastructure, tools, and modes of operation revealed a potential relationship between Ajax Security Team and the campaign Operation Woolen-Goldfish.(Citation: Check Point Rocket Kitten)(Citation: TrendMicro Operation Woolen Goldfish March 2015)'},
  {'source_name': 'AjaxTM',
   'description': '(Citation: FireEye Operation Saffron Rose 2013)'},
  {'source_name': 'Rocket Kitten',
   'description': 'Analysis of infrastructure, tools, and modes of operation revealed a potential relationship between Ajax Security Team and Rocket Kitten.(Citation: Check Point Rocket Kitten)(Citation: IranThreats Kittens Dec 2017)'},
  {'source_

# Retrive all Techniques used by all Groups - (Automatic)
-----------------------

In [19]:
from attackcti import attack_client
lift = attack_client()
%time techniques_used = lift.get_techniques_used_by_all_groups()

CPU times: user 7.33 s, sys: 148 ms, total: 7.48 s
Wall time: 11.8 s


In [20]:
len(techniques_used)

2512

In [21]:
techniques_used[0]

{'created_by_ref': 'identity--c78cb6e5-0c4b-4611-8297-d1b8b55e40b5',
 'object_marking_refs': ['marking-definition--fa42a846-8d90-4e51-bc29-71d5b4802168'],
 'external_references': [{'external_id': 'G0130',
   'source_name': 'mitre-attack',
   'url': 'https://attack.mitre.org/groups/G0130'},
  {'source_name': 'Operation Woolen-Goldfish',
   'description': 'Analysis of infrastructure, tools, and modes of operation revealed a potential relationship between Ajax Security Team and the campaign Operation Woolen-Goldfish.(Citation: Check Point Rocket Kitten)(Citation: TrendMicro Operation Woolen Goldfish March 2015)'},
  {'source_name': 'AjaxTM',
   'description': '(Citation: FireEye Operation Saffron Rose 2013)'},
  {'source_name': 'Rocket Kitten',
   'description': 'Analysis of infrastructure, tools, and modes of operation revealed a potential relationship between Ajax Security Team and Rocket Kitten.(Citation: Check Point Rocket Kitten)(Citation: IranThreats Kittens Dec 2017)'},
  {'source_

# Create Navigator Group Layer Files - (Manual)
-----------------------

## Create a list of group dictionaries
To make things easier, I create a list of dictionaries where each group name is the main key and the value is a list where I append every single technique involving that group. I get that information from the `get_techniques_used_by_all_groups()` results.

In [22]:
groups = lift.get_groups()
groups = lift.remove_revoked(groups)
groups_list = []
for g in groups:
    group_dict = dict()
    group_dict[g['name']] = []
    groups_list.append(group_dict)
groups_list[89]

{'Lazarus Group': []}

## Group techniques by group
We can then use the output of the `get_techniques_used_by_all_groups()` function and start appending techniques to the dictionaries with the key name that matches the group name being involved with each technique.

In [23]:
for group in groups_list:
    for group_name,techniques_list in group.items():
        for gut in techniques_used:
            if group_name == gut['name']:
                technique_dict = dict()
                technique_dict['techniqueId'] = gut['technique_id']
                technique_dict['techniqueName'] = gut['technique']
                technique_dict['comment'] = gut['relationship_description']
                technique_dict['tactic'] = gut['tactic']
                technique_dict['group_id'] = gut['external_references'][0]['external_id']
                techniques_list.append(technique_dict)
groups_list[89]

{'Lazarus Group': [{'techniqueId': 'T1036.005',
   'techniqueName': 'Match Legitimate Name or Location',
   'comment': "[Lazarus Group](https://attack.mitre.org/groups/G0032) has renamed the [TAINTEDSCRIBE](https://attack.mitre.org/software/S0586) main executable to disguise itself as Microsoft's narrator.(Citation: CISA MAR-10288834-2.v1  TAINTEDSCRIBE MAY 2020)",
   'tactic': [KillChainPhase(kill_chain_name='mitre-attack', phase_name='defense-evasion')],
   'group_id': 'G0032'},
  {'techniqueId': 'T1583.006',
   'techniqueName': 'Web Services',
   'comment': '[Lazarus Group](https://attack.mitre.org/groups/G0032) has hosted malicious downloads on Github.(Citation: CISA AppleJeus Feb 2021)',
   'tactic': [KillChainPhase(kill_chain_name='mitre-attack', phase_name='resource-development')],
   'group_id': 'G0032'},
  {'techniqueId': 'T1588.004',
   'techniqueName': 'Digital Certificates',
   'comment': '[Lazarus Group](https://attack.mitre.org/groups/G0032) has obtained SSL certificates 

## Run dynamic navigator layer template

In [24]:
import json
for group in groups_list:
    for k,v in group.items():
        if v:
            actor_layer = {
                "description": ("Enterprise techniques used by {0}, ATT&CK group {1} v1.0".format(k,v[0]['group_id'])),
                "name": ("{0} ({1})".format(k,v[0]['group_id'])),
                "domain": "mitre-enterprise",
                "version": "2.2",
                "techniques": [
                    {
                        "score": 1,
                        "techniqueID" : technique['techniqueId'],
                        "techniqueName" : technique['techniqueName'],
                        "comment": technique['comment']
                    } for technique in v
                ],
                "gradient": {
                    "colors": [
                        "#ffffff",
                        "#ff6666"
                    ],
                    "minValue": 0,
                    "maxValue": 1
                },
                "legendItems": [
                    {
                        "label": ("used by {}".format(k)),
                        "color": "#ff6666"
                    }
                ]
            }
            with open(('{0}_{1}.json'.format(k,v[0]['group_id'])), 'w') as f:
                f.write(json.dumps(actor_layer))

In [25]:
! ls *.json

ALLANITE_G1000.json           Indrik Spider_G0119.json
APT-C-36_G0099.json           Ke3chang_G0004.json
APT12_G0005.json              Kimsuky_G0094.json
APT16_G0023.json              Lazarus Group_G0032.json
APT17_G0025.json              Leafminer_G0077.json
APT18_G0026.json              Leviathan_G0065.json
APT19_G0073.json              Machete_G0095.json
APT1_G0006.json               Magic Hound_G0059.json
APT28_G0007.json              Moafee_G0002.json
APT29_G0016.json              Mofang_G0103.json
APT30_G0013.json              Molerats_G0021.json
APT32_G0050.json              MuddyWater_G0069.json
APT33_G0064.json              Mustang Panda_G0129.json
APT37_G0067.json              Naikon_G0019.json
APT38_G0082.json              Night Dragon_G0014.json
APT39_G0087.json              OilRig_G0049.json
APT3_G0022.json               Operation Wocao_G0116.json
APT41_G0096.json              Orangeworm_G0071.json
Ajax Security Team_G0130.json PLATINUM_G0068.json
Axiom_G0001.json         

We can delete all the files with the following command.

In [26]:
! rm *.json

# Create Navigator Group Layer Files - (Automatic)
-----------------------

In [27]:
from attackcti import attack_client
lift = attack_client()

%time lift.export_groups_navigator_layers()

CPU times: user 8.02 s, sys: 215 ms, total: 8.23 s
Wall time: 12.8 s


In [28]:
! ls *.json

ALLANITE_G1000.json           Indrik Spider_G0119.json
APT-C-36_G0099.json           Ke3chang_G0004.json
APT12_G0005.json              Kimsuky_G0094.json
APT16_G0023.json              Lazarus Group_G0032.json
APT17_G0025.json              Leafminer_G0077.json
APT18_G0026.json              Leviathan_G0065.json
APT19_G0073.json              Machete_G0095.json
APT1_G0006.json               Magic Hound_G0059.json
APT28_G0007.json              Moafee_G0002.json
APT29_G0016.json              Mofang_G0103.json
APT30_G0013.json              Molerats_G0021.json
APT32_G0050.json              MuddyWater_G0069.json
APT33_G0064.json              Mustang Panda_G0129.json
APT37_G0067.json              Naikon_G0019.json
APT38_G0082.json              Night Dragon_G0014.json
APT39_G0087.json              OilRig_G0049.json
APT3_G0022.json               Operation Wocao_G0116.json
APT41_G0096.json              Orangeworm_G0071.json
Ajax Security Team_G0130.json PLATINUM_G0068.json
Axiom_G0001.json         

We can delete all the files with the following command.

In [29]:
! rm *.json